In [17]:
import pandas as pd
import datetime
import plotly.express as px
import plotly.graph_objects as go

In [18]:
def wrangle(file,services=10):
    df = pd.read_csv(file, parse_dates=["date"], index_col="date")
    df = df[~df.index.isna()]

    df["zipcode"] = [i.split()[-3].rstrip(",") for i in df["address"]]
    zips = [i.isnumeric() for i in df["zipcode"]]
    df = df[(zips)]
    df["zipcode"] = df["zipcode"].astype("int64")

    df["month"] = [i.strftime("%Y-%m") for i in df.index]

    top_services = list(df["service_description"].value_counts()[:services].index)
    for i,v in enumerate(df["service_description"]):
        if v not in top_services:
            df["service_description"][i] = "Other"

    return df
data = wrangle("../data/mock_data.csv",10)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 996 entries, 2018-05-14 to 2018-03-27
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   client_id            996 non-null    int64  
 1   full_name            996 non-null    object 
 2   gender               996 non-null    object 
 3   age                  996 non-null    int64  
 4   service_description  996 non-null    object 
 5   address              996 non-null    object 
 6   latitude             996 non-null    float64
 7   longitude            996 non-null    float64
 8   zipcode              996 non-null    int64  
 9   month                996 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 85.6+ KB


/var/folders/7h/7j7cb5_n36171bhxn4yd3s4c0000gn/T/ipykernel_42317/2761684952.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
print(data.shape)
data.head()

(996, 10)


,client_id,full_name,gender,age,service_description,address,latitude,longitude,zipcode,month
date,,,,,,,,,,
2018-05-14,54892,Mehetabel Aylmore,Male,20,Case Management,"Gilman Dr W, Seattle, WA 98119, United States",47.636289,-122.371025,98119,2018-05
2018-05-14,49499,Elga Raeburn,Female,9,Case Management,"7th Avenue West, Seattle, WA 98119, United States",47.639123,-122.365667,98119,2018-05
2018-05-21,64034,Kimmie Richardot,Female,7,Case Management,"West Lee Street, Seattle, WA 98119, United States",47.629724,-122.369483,98119,2018-05
2018-09-17,42222,Adham Polack,Female,62,Case Management,"8th Avenue West, Seattle, WA 98119, United States",47.638473,-122.369279,98119,2018-09
2018-05-21,45792,Jeff Winspar,Female,8,Other,"14th Ave W, Seattle, WA 98119, United States",47.632918,-122.372471,98119,2018-05


In [20]:
def service_splits(categories,demographic,norm=False):
    """
    Splits a categorical series and finds the frequency of each unique
    value crossed with the values of a specified demographic series.
    Returns demographic counts of each service into a bar graph.
    """
    return px.bar(pd.crosstab(categories, demographic),
                  labels={"service_description": "Services",
                          "value": "Counts",
                          demographic.name: demographic.name.capitalize()})

services = data["service_description"]
demo = data["gender"] 
service_splits(services,demo)

In [21]:
def service_splits_demographic(categories,demographic,norm=False):
    """
    Splits a categorical series and finds the frequency of each unique
    value crossed with the values of a specified demographic series.
    Returns demographic counts of each service into a bar graph.
    """
    genders = ['Male', 'Female', 'Nonbinary', 'Other']
    if demographic in genders:
        mask = data['gender'] == demographic
        df = data[mask] 
        df_categories = df[categories]
        df_demographic = df['gender']

        fig = px.bar(pd.crosstab(df_categories, df_demographic),
                     labels={"service_description": "Services",
                              "value": "Counts"},
                     title= "Service Counts: {}".format(demographic))

        return fig.update_layout(showlegend=False)
    
    if len(str(demographic)) < 3:
        mask = data['age'] == demographic
        df = data[mask]
        df_categories = df[categories]
        df_demographic = df['age']

        fig = px.bar(pd.crosstab(df_categories, df_demographic),
                     labels={"service_description": "Services",
                              "value": "Counts"},
                     title= "Service Counts for {} Year Olds"
                            .format(str(demographic)))

        return fig.update_layout(showlegend=False)
    
    if len(str(demographic)) > 3:
        mask = data['client_id'] == demographic
        df = data[mask]
        df_categories = df[categories]
        df_demographic = df['client_id']

        fig = px.bar(pd.crosstab(df_categories, df_demographic),
                     labels={"service_description": "Services",
                              "value": "Counts"},
                     title= "Service Counts for Client: {}"
                            .format(str(demographic)))

        return fig.update_layout(showlegend=False)
    
service_splits_demographic('service_description', 'Male')

In [22]:
def service_splits_gender(services='None',genders= 'None',norm=False):
    """
    Splits a categorical series and finds the frequency of each unique
    value crossed with the values of a specified demographic series.
    Returns demographic counts of selected service and genders into a bar graph.
    Both serives and genders must be a list.
    """
    frames = []
    for i in services:
        df1 = data[data['service_description'] == i]
        frames.append(df1)
    hold = pd.concat(frames, ignore_index=True)
    frames2 = []
    for j in genders:
        df2 = hold[hold['gender'] == j]
        frames2.append(df2)
    df = pd.concat(frames2, ignore_index=True)
    fig = px.bar(pd.crosstab(df['service_description'],
                             df['gender']),
                 labels={"service_description": "Services",
                          "value": "Counts"},
                 title= "Service Counts by Gender")
    fig.update_layout(uniformtext_minsize=8,
                      uniformtext_mode='hide',
                      barmode='group',
                      bargap=0.15, 
                      bargroupgap=0.1)
    return fig.show()
services = ['Food', 'Bus Pass', 'Bus Tokens', 'Other', 'Mental Health Counseling']
genders = ['Male', 'Female', 'Nonbinary']
service_splits_gender(services, genders)

In [23]:
def services_by_age(ages):
    """
    Splits a series of ages into age groups,
    gets the frequency of each age,
    then plots it into a pie chart.
    """
    age_groups = {}
    age_groups["<18"]   = (ages < 18).sum()
    age_groups["18-24"] = ((ages >= 18) & (ages < 25)).sum()
    age_groups["25-34"] = ((ages >= 25) & (ages < 35)).sum()
    age_groups["35-44"] = ((ages >= 35) & (ages < 45)).sum()
    age_groups["45-54"] = ((ages >= 45) & (ages < 55)).sum()
    age_groups["55+"]   = (ages >= 55).sum()

    age_groups = pd.Series(age_groups)

    fig = px.pie(age_groups,
                 values=age_groups,
                 names=age_groups.index,
                 labels={"index": "Age Group"},
                 title="Service Counts by Age Group",
                 )
    fig.update_traces(textinfo='percent+label')

    return fig.show()

services_by_age(data["age"])

In [24]:
fig = px.bar(pd.crosstab(data["month"],data["service_description"]),
             labels={"month": "Month", "value": "Service Counts", "service_description": "Services"},
             title="Service Frequency by Month")
fig.show()

In [27]:
def service_counts_by_month(services='None',
                            dates='None', norm=False):
    """
    Splits service categories by month and displays them as a bar graph. Need to explore
    how to receive user input and push to BE.
    """
    frames = []
    for i in services:
        df1 = data[data['service_description'] == i]
        frames.append(df1)
    hold = pd.concat(frames, ignore_index=True)
    frames2 = []
    for j in dates:
        df2 = hold[hold['month'] == j]
        frames2.append(df2)
    df = pd.concat(frames2, ignore_index=True)
    fig = px.bar(pd.crosstab(df['month'],
                             df['service_description']),
                 labels={"month": "Month",
                          "value": "Counts"},
                 title= "Service Counts by Month")
    fig.update_layout(barmode='group',
                      bargap=0.15, 
                      bargroupgap=0.1,
                      xaxis_tickangle=-45,
                      yaxis=dict(
                          titlefont_size=16,
                          tickfont_size=14,))
    return fig.show()
services = ['Bus Pass', 'Bus Tokens', 'Food', 'Case/Care management']
months = ['2018-01','2018-02','2018-03','2018-04','2018-05','2018-06','2018-07','2018-08','2018-09','2018-10','2018-11','2018-12',
         '2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11','2019-12',
         '2020-01']
service_counts_by_month(services, months)

In [14]:
# pd.Series([i.split(", ")[1] for i in data["address"]])

In [15]:
map = pd.read_csv("../data/services_by-zipcode.csv")
map.head()

,zipcode,counts,city,latitude,longitude
0,99201,16,Spokane,47.663945,-117.43185
1,99217,12,Spokane,47.707542,-117.33776
2,99021,9,Mead,47.811080,-117.22306
3,99037,8,Spokane Valley,47.643346,-117.19993
4,99202,7,Spokane,47.656692,-117.37899


In [16]:
fig = px.scatter_mapbox(map, lat="latitude", lon='longitude',
                     color="city", # which column to use to set the color of markers
                     hover_name="zipcode", # column added to hover information
                     size='counts',
                     zoom=10,
                     )

fig.update_layout(mapbox_style="open-street-map")
fig.show()